### Pytorch를 사용하여 배깅을 CNN에 적용하는 실습 : CIFAR10 데이터셋 활용

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

c:\Miniconda\envs\Pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class CNN(nn.Module) :
    def __init__(self) :
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(32 * 8 * 8, 10)
    
    def forward(self, x) :
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [4]:
train_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.AutoAugment(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.3,0.3,0.3))
])

test_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.3,0.3,0.3))
    
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=False, transform=train_transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=False, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
# Create an ensemble of CNN model
num_models = 5 
models = [CNN().to(device) for _ in range(num_models)]
print(models)
# Define loss function and optimizer 
criterion = nn.CrossEntropyLoss()
optimizers = [optim.AdamW(model.parameters(), lr=0.001) for model in models]

[CNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=2048, out_features=10, bias=True)
), CNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=2048, out_features=10, bias=True)
), CNN(
  (features): Sequential(
    (0): Conv2d(3, 16, k

In [6]:
num_epochs = 2
for epoch in range(num_epochs) : 
    for model, optimizer in zip(models, optimizers) :
        model.train()
        for images, labels in train_loader :
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], loss : {loss}")            
    # Evaluation after each epoch
    for model in models :
        model.eval()
    predictions = []
    targets = []
    with torch.no_grad() :
        for images, labels in test_loader :
            images = images.to(device)
            labels = labels.to(device)
            ensemble_outputs = torch.zeros((images.size(0), 10)).to(device)
            for model in models :
                outputs = model(images)
                ensemble_outputs += outputs/num_models
                _, pred = torch.max(ensemble_outputs.data, 1)
            predictions.extend(pred.cpu().numpy())
            targets.extend(labels.cpu().numpy())

    accuracy = accuracy_score(targets, predictions)
    print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy : {accuracy}")

Epoch [1/2], loss : 1.7271630764007568
Epoch [1/2], loss : 1.722511887550354
Epoch [1/2], loss : 2.1370348930358887
Epoch [1/2], loss : 1.8993076086044312
Epoch [1/2], loss : 1.8842761516571045
Epoch [1/2], Accuracy : 0.532
Epoch [2/2], loss : 1.4198983907699585
Epoch [2/2], loss : 1.1939564943313599
Epoch [2/2], loss : 1.2973308563232422
Epoch [2/2], loss : 1.1867121458053589
Epoch [2/2], loss : 2.0314269065856934
Epoch [2/2], Accuracy : 0.576
